In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tabula

In [3]:
UT_pdf = os.path.join(os.path.dirname(os.getcwd()), '../Data/Raw_data/UT Measurements_Report.pdf')

In [295]:
#TO DO p7_table by hand

In [5]:
tables_scatter_atten = tabula.read_pdf(UT_pdf, pages=[8, 29,34], lattice=True)

In [8]:
#PIPE backscatter
table_0 = tables_scatter_atten[0].T.reset_index(drop=True).T 
table_0.rename(columns = {0:'Sample',
                          1:'backscatter_rms'}, inplace=True)
table_0.to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Intermediate_data/'+'PIPE_ut_backscatter.csv'))

In [242]:
#TUBE attenuation
tables_scatter_atten[1].rename(columns = {'Unnamed: 0':'Sample'}, inplace=True)
tables_scatter_atten[1].to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Intermediate_data/'+'TUBE_ut_attenuation.csv'))

In [251]:
#PIPE attenuation
tables_scatter_atten[2].rename(columns = {'Unnamed: 0':'Sample'}, inplace=True)
tables_scatter_atten[2].to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Intermediate_data/'+'PIPE_ut_attenuation.csv'))

In [17]:
def clean_abs_col(df, suffix=None, avg_pow_10=1000, std_pow_10=10000):
    '''
    remove useless row and calculate the final data
    Args:
     - df: data extracted from tabula
     - suffix: time window ('str')
     - avg_pow_10 : int to divide average value
     - std_pow_10 : int to divide SD value
    return df
    '''
    df.drop(0, inplace=True)
    df[['Absorption_avg', 'Absorption_std']] = df[['Absorption_avg', 'Absorption_std']].astype('float')
    df['Absorption_avg'] = df['Absorption_avg']/avg_pow_10
    df['Absorption_std'] = df['Absorption_std']/std_pow_10 
    df.set_index('Sample', inplace=True)
    df = df.add_suffix(suffix)
    print(df)
    return df

In [18]:
tables = tabula.read_pdf(UT_pdf, pages=[54, 55, 56, 57, 58, 60, 61], lattice=True)

In [19]:
#TUBE absorp 50s
tables[0].rename(columns = {'Value of internal friction Q-1':'Absorption_avg',
                           'Unnamed: 0':'Absorption_std'}, inplace=True)
tube_ut_50 = clean_abs_col(tables[0], '_50')
tube_ut_50.to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Intermediate_data/'+'TUBE_ut_absorption_50us.csv'))

                       Absorption_avg_50  Absorption_std_50
Sample                                                     
As received                     0.001257           0.000210
Fully ferritic                  0.003377           0.000817
HAZ                             0.001693           0.000460
HAZ + Tempered                  0.001390           0.000418
Normalized                      0.003128           0.001147
Normalized + Tempered           0.001838           0.000622
Over-tempered                   0.001526           0.000433
Tempered                        0.001611           0.000873


In [20]:
#TUBE_blind absorp 50s
table_50BT = pd.concat([tables[1], tables[2].T.reset_index(drop=False).T ])
table_50BT.rename(columns = {0:'Sample',
                          1:'Absorption_avg',
                           2:'Absorption_std'}, inplace=True)
table_50BT = table_50BT.iloc[:, 0:3]
table_50BT = clean_abs_col(table_50BT, '_50')
table_50BT.reset_index(drop=True, inplace=True)
table_50BT.to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Intermediate_data/'+'TUBE_blind_ut_absorption_50us.csv'))

        Absorption_avg_50  Absorption_std_50
Sample                                      
B1               0.001264           0.000519
B3               0.001168           0.000555
B4               0.002084           0.000648
B5               0.001141           0.000447
B6               0.001323           0.000574
B7               0.001908           0.000926
B8               0.000933           0.000304


In [22]:
#TUBE absorp 100s
tables[3].rename(columns = {'Value of internal friction Q-1':'Absorption_avg',
                           'Unnamed: 0':'Absorption_std'}, inplace=True)
tube_ut_100 = clean_abs_col(tables[3], '_100', 10000, 10000)
tube_ut_100.to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Intermediate_data/'+'TUBE_ut_absorption_100us.csv'))

                       Absorption_avg_100  Absorption_std_100
Sample                                                       
As received                      0.000982            0.000325
Fully ferritic                   0.000866            0.000276
HAZ                              0.001051            0.000437
HAZ + Tempered                   0.000788            0.000317
Normalized                       0.000959            0.000175
Normalized + Tempered            0.001287            0.000330
Over-tempered                    0.000780            0.000235
Tempered                         0.000742            0.000287


In [23]:
#TUBE absorp 200s
tables[5].rename(columns = {'Value of internal friction Q-1':'Absorption_avg',
                           'Unnamed: 0':'Absorption_std'}, inplace=True)
tube_ut_200 = clean_abs_col(tables[5], '_200', 10000, 100000)
tube_ut_200.to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Intermediate_data/'+'TUBE_ut_absorption_200us.csv'))

                       Absorption_avg_200  Absorption_std_200
Sample                                                       
As received                      0.000488            0.000081
Fully ferritic                   0.000657            0.000088
HAZ                              0.000425            0.000066
HAZ + Tempered                   0.000494            0.000176
Normalized                       0.000657            0.000062
Normalized + Tempered            0.000835            0.000165
Over-tempered                    0.000476            0.000114
Tempered                         0.000384            0.000041


In [24]:
#TUBE_blind absorp 200s
table_200BT = pd.concat([tables[6].T.reset_index(drop=False).T, tables[7].T.reset_index(drop=False).T ])
table_200BT.rename(columns = {0:'Sample',
                          1:'Absorption_avg',
                           2:'Absorption_std'}, inplace=True)
table_200BT = table_200BT.iloc[1:]
table_200BT = clean_abs_col(table_200BT,  '_200', 10000, 100000)
table_200BT.reset_index(drop=True, inplace=True)
table_200BT.to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Intermediate_data/'+'TUBE_blind_ut_absorption_200us.csv'))

        Absorption_avg_200  Absorption_std_200
Sample                                        
B1                0.000373            0.000094
B2                0.000874            0.000234
B3                0.000349            0.000078
B4                0.000701            0.000136
B5                0.000397            0.000088
B6                0.000401            0.000110
B8                0.000416            0.000091


In [25]:
#TUBE absorp 500s
tables[8].rename(columns = {'Value of internal friction Q-1':'Absorption_avg',
                           'Unnamed: 0':'Absorption_std'}, inplace=True)
tube_ut_500 = clean_abs_col(tables[8],  '_500', 10000, 100000)
tube_ut_500.to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Intermediate_data/'+'TUBE_ut_absorption_500us.csv'))

                       Absorption_avg_500  Absorption_std_500
Sample                                                       
As received                      0.000283            0.000023
Fully ferritic                   0.000493            0.000064
HAZ                              0.000349            0.000027
HAZ + Tempered                   0.000355            0.000111
Normalized                       0.000443            0.000075
Normalized + Tempered            0.000530            0.000066
Over-tempered                    0.000332            0.000031
Tempered                         0.000283            0.000028


In [26]:
#TUBE_blind absorp 500s
table_500BT = pd.concat([tables[9].T.reset_index(drop=False).T, tables[10].T.reset_index(drop=False).T ])
table_500BT.rename(columns = {0:'Sample',
                          1:'Absorption_avg',
                           2:'Absorption_std'}, inplace=True)
table_500BT = table_500BT.iloc[1:]
table_500BT = clean_abs_col(table_500BT,  '_500', 10000, 100000)
table_500BT.reset_index(drop=True, inplace=True)
table_500BT.to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Intermediate_data/'+'TUBE_blind_ut_absorption_500us.csv'))

        Absorption_avg_500  Absorption_std_500
Sample                                        
B1                0.000233            0.000018
B2                0.000571            0.000139
B4                0.000379            0.000100
B5                0.000230            0.000033
B6                0.000240            0.000026
B7                0.000406            0.000084
B8                0.000319            0.000076


In [28]:
#PIPE absorp 50s
tables[12].rename(columns = {'Value of internal friction Q-1':'Absorption_avg',
                           'Unnamed: 0':'Absorption_std'}, inplace=True)
pipe_ut_50 = clean_abs_col(tables[12], '_50')
pipe_ut_50.to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Intermediate_data/'+'PIPE_ut_absorption_50us.csv'))

                       Absorption_avg_50  Absorption_std_50
Sample                                                     
As received                     0.002499           0.000645
Ferritic                        0.001802           0.000786
HAZ                             0.001627           0.000437
HAZ + Tempered                  0.001935           0.000359
Normalized                      0.002016           0.000674
Normalized + Tempered           0.002380           0.000794
Over-tempered                   0.001690           0.000702
Tempered                        0.002338           0.000497


In [104]:
#PIPE absorp 100s
tables[13].rename(columns = {'Value of internal friction Q-1':'Absorption_avg',
                           'Unnamed: 0':'Absorption_std'}, inplace=True)
pipe_ut_100 = clean_abs_col(tables[13])
pipe_ut_100.to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Intermediate_data/'+'PIPE_ut_absorption_100us.csv'))